In [157]:
import pandas as pd
import numpy as np
from functools import reduce
from bokeh.io import show, output_notebook, output_file
from bokeh.plotting import figure
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool, LabelSet, ColumnDataSource
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4

output_notebook()

Loading BokehJS ...

In [65]:
irs_org_list = np.unique(irs_990['name_org'])
nysdos_overlap = nysdos[nysdos['Current Entity Name'].isin(irs_org_list)]
checkbook_overlap = checkbook[checkbook['Prime Vendor'].isin(irs_org_list)]

In [66]:
nysdos_overlap = nysdos_overlap.rename(columns={'Current Entity Name':'name_org'})
checkbook_overlap = checkbook_overlap.rename(columns={'Prime Vendor':'name_org'})

In [67]:
df_list = [irs_990,nysdos_overlap,checkbook_overlap]
full_data = reduce(lambda left, right: pd.merge(left, right, on='name_org', how='left'), df_list)

In [68]:
checkbook_subset = checkbook[['Prime Vendor','Prime Contract Original Amount',
                              'Prime Contracting Agency','Prime Contract Industry']].copy()

irs_subset = irs_990[['ein', 'name_org',
                      'name_contractor_1','amt_paid_contractor_1','services_contractor_1',
                      'name_contractor_2','amt_paid_contractor_2','services_contractor_2',
                      'name_contractor_3','amt_paid_contractor_3','services_contractor_3',
                      'name_contractor_4','amt_paid_contractor_4','services_contractor_4',
                      'name_contractor_5','amt_paid_contractor_5','services_contractor_5']].copy()

irs_mini = irs_990[['ein','name_org']].copy()

In [71]:
agencies = list(set(checkbook['Prime Contracting Agency']))
ids = np.random.randint(low=10000, high=30000, size=len(agencies))
agencies_dict = dict(zip(agencies, ids))
checkbook_subset['id'] = checkbook_subset['Prime Contracting Agency'].map(agencies_dict)

In [188]:
checkbook_subset.head()

,Prime Vendor,Prime Contract Original Amount,Prime Contracting Agency,Prime Contract Industry,id
0,CONSOLIDATED EDISON COMPANY OF NEW YORK INC,21410.00,Department of Transportation,Not Classified,19880
1,NEW YORK CITY ECONOMIC DEVELOPMENT CORPORATION,49000.00,Department of Small Business Services,Construction Services,18593
2,NEW YORK CITY ECONOMIC DEVELOPMENT CORPORATION,7488848.95,Department of Small Business Services,Construction Services,18593
3,NEW YORK CITY ECONOMIC DEVELOPMENT CORPORATION,95825.00,Department of Small Business Services,Standardized Services,18593
4,NEW YORK CITY ECONOMIC DEVELOPMENT CORPORATION,425010.00,Department of Small Business Services,Construction Services,18593


In [122]:
irs_contractor1 = irs_subset[['ein','name_org','name_contractor_1','amt_paid_contractor_1','services_contractor_1']].copy()
irs_contractor2 = irs_subset[['ein','name_org','name_contractor_2','amt_paid_contractor_2','services_contractor_2']].copy()
irs_contractor3 = irs_subset[['ein','name_org','name_contractor_3','amt_paid_contractor_3','services_contractor_3']].copy()
irs_contractor4 = irs_subset[['ein','name_org','name_contractor_4','amt_paid_contractor_4','services_contractor_4']].copy()
irs_contractor5 = irs_subset[['ein','name_org','name_contractor_5','amt_paid_contractor_5','services_contractor_5']].copy()

irs_contractor1 = irs_contractor1.rename(columns={'ein':'id',
                                                  'name_contractor_1':'name_contractor',
                                                  'amt_paid_contractor_1':'amt_paid_contractor',
                                                 'services_contractor_1':'services_contractor'})

irs_contractor2 = irs_contractor2.rename(columns={'ein':'id',
                                                  'name_contractor_2':'name_contractor',
                                                  'amt_paid_contractor_2':'amt_paid_contractor',
                                                 'services_contractor_2':'services_contractor'})

irs_contractor3 = irs_contractor3.rename(columns={'ein':'id',
                                                  'name_contractor_3':'name_contractor',
                                                  'amt_paid_contractor_3':'amt_paid_contractor',
                                                 'services_contractor_3':'services_contractor'})

irs_contractor4 = irs_contractor4.rename(columns={'ein':'id',
                                                  'name_contractor_4':'name_contractor',
                                                  'amt_paid_contractor_4':'amt_paid_contractor',
                                                 'services_contractor_4':'services_contractor'})

irs_contractor5 = irs_contractor5.rename(columns={'ein':'id',
                                                  'name_contractor_5':'name_contractor',
                                                  'amt_paid_contractor_5':'amt_paid_contractor',
                                                 'services_contractor_5':'services_contractor'})

irs_contractor_clean = pd.concat([irs_contractor1, irs_contractor2, irs_contractor3, irs_contractor4, irs_contractor5])

In [123]:
irs_contractor_clean = irs_contractor_clean.drop(['services_contractor'],axis=1)
irs_contractor_clean.head()

,id,name_org,name_contractor,amt_paid_contractor
0,133783732,Healthfirst PHSP Inc,CAREMARK LLC,1.333975e+09
1,113029569,Healthfirst Health Plan Inc,CAREMARK LLC,7.669487e+08
2,131623965,College Entrance Examination Board,EDUCATIONAL TESTING SERVICES,3.591001e+08
3,131924236,Memorial Sloan-Kettering Cancer Center,TURNER CONSTRUCTION,2.279733e+08
4,912154267,Memorial Sloan-Kettering Cancer Center,TURNER CONSTRUCTION,2.279733e+08


In [126]:
checkbook_irs = pd.merge(checkbook_subset, irs_mini, how='inner', left_on='Prime Vendor', right_on='name_org')
checkbook_irs = checkbook_irs.drop(['Prime Vendor','ein','Prime Contract Industry'],axis=1)
checkbook_irs = checkbook_irs.rename(columns={'name_org':'name_contractor','Prime Contracting Agency':'name_org',
                                              'Prime Contract Original Amount':'amt_paid_contractor'})
checkbook_irs.head()

,amt_paid_contractor,name_org,id,name_contractor
0,1.0,Department of Social Services,17524,WEST HARLEM GROUP ASSISTANCE INC
1,10000.0,Department of Youth and Community Development,22427,WEST HARLEM GROUP ASSISTANCE INC
2,125000.0,Department of Youth and Community Development,22427,WEST HARLEM GROUP ASSISTANCE INC
3,7500.0,Department of Youth and Community Development,22427,WEST HARLEM GROUP ASSISTANCE INC
4,50000.0,Housing Preservation and Development,12827,WEST HARLEM GROUP ASSISTANCE INC


In [134]:
checkbook_irs_amts = checkbook_irs.groupby(['id','name_org','name_contractor'], as_index=False)['amt_paid_contractor'].sum()

In [144]:
edges = pd.concat([checkbook_irs_amts,irs_contractor_clean])
print(len(edges),len(checkbook_irs_amts),len(irs_contractor_clean))

27010 440 26570


In [147]:
nodes = checkbook_irs_amts[['id','name_org']].copy().drop_duplicates()
nodes.head()

,id,name_org
0,10757,Department of Parks and Recreation
8,10892,Department of Education
84,12148,Department of Sanitation
86,12827,Housing Preservation and Development
94,13245,Mayoralty


In [216]:
g = nx.from_pandas_edgelist(checkbook_irs_amts, source='name_org', target='name_contractor', edge_attr='amt_paid_contractor')

plot = figure(title="Nonprofit Network Map", x_range=(-1.1,1.1), y_range=(-1.1,1.1))
node_hover_tool = HoverTool(tooltips=[('Org Name', '@name_org')])
plot.add_tools(node_hover_tool)

graph = from_networkx(g, nx.spring_layout, scale=3, center=(0,0))

graph.node_renderer.glyph = Circle(size=10, fill_color=Spectral4[1])
graph.edge_renderer.glyph = MultiLine(line_color='amt_paid_contractor', line_alpha=0.8, line_width=1.5)

graph.selection_policy = NodesAndLinkedEdges()

plot.renderers.append(graph)

show(plot)